In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import sys
sys.path.append('../')
from BirdCard import BirdCard
from imagepaths import habitat_paths, food_paths, nest_paths
from load_data_functions import load_birdcards
from Gameplay_Constants import FOOD_TYPES, BIRDCARD_FOOD_MAPPING

from xml.dom.minidom import getDOMImplementation, parseString
import os

WORKDIR = r'C:\Users\osmou\Documents\GitHub\wingspan-bird-brain\src'

birdcards = load_birdcards()

# Helpers

In [3]:
def imgTag(src, width=20, height=20):
    return parseString(f'<img src="{src}" width="{width}" height="{height}"></img>').documentElement

# Creators

In [4]:
#=======================================================================================================================
def createBirdCardDiv(birdcard):
    assert type(birdcard) is BirdCard

    topDiv = parseString(f'<div class="birdcard" style="border: 1px solid black; font-family: cursive; font-weight: bold; display: inline-block;"> <u>{birdcard.common_name}</u> </div>').documentElement
    topDiv.appendChild(parseString(f'<div class="bird info" style="font-weight: normal;">VP: {birdcard.victory_points}, Wingspan: {birdcard.wingspan}</div>').documentElement)

    # Habitats
    habitatDiv = parseString('<div class="habitats"></div>').documentElement
    if birdcard.forest:
        habitatDiv.appendChild(imgTag(os.path.join(WORKDIR, habitat_paths['Forest'])))
    if birdcard.grassland:
        habitatDiv.appendChild(imgTag(os.path.join(WORKDIR, habitat_paths['Grassland'])))
    if birdcard.wetland:
        habitatDiv.appendChild(imgTag(os.path.join(WORKDIR, habitat_paths['Wetland'])))
    topDiv.appendChild(habitatDiv)

    # Food Cost
    sep = '/' if birdcard.slash_food_cost else '+'
    totalFoodCost = birdcard.total_food_cost
    foodAdded = 0
    foodCostDiv = parseString('<div class="foodCost" style="display: flex;"></div>').documentElement
    for food in FOOD_TYPES:
        cnt = birdcard.__dict__[BIRDCARD_FOOD_MAPPING[food]]
        for i in range(cnt):
            foodCostDiv.appendChild(imgTag(os.path.join(WORKDIR, food_paths[food])))
            foodAdded += 1
            if foodAdded < totalFoodCost:
                # foodCostDiv.appendChild(doc.createTextNode(sep))
                foodCostDiv.appendChild(parseString(f'<div>{sep}</div>').documentElement)
    topDiv.appendChild(foodCostDiv)

    # Nest type and eggs
    nestDiv = parseString('<div class="nest"></div>').documentElement
    nestDiv.appendChild(imgTag(os.path.join(WORKDIR, nest_paths[birdcard.nest_type])))
    for i in range(birdcard.laideggs):
        nestDiv.appendChild(imgTag(os.path.join(WORKDIR, 'ui', 'images', 'game_egg.png'), height=25, width=25))
    for i in range(birdcard.egg_capacity - birdcard.laideggs):
        nestDiv.appendChild(imgTag(os.path.join(WORKDIR, 'ui', 'images', 'game_smallegg.png')))
    topDiv.appendChild(nestDiv)

    # Power text
    topDiv.appendChild(parseString(f'<div class="power text" style="font-weight: normal;">{birdcard.power_text}</div>').documentElement)

    return topDiv

In [5]:
# Create blank HTML
impl = getDOMImplementation()
doc = impl.createDocument(None, 'gui', None)
html = doc.documentElement

# Add test birdcard
birdcard = birdcards[0]
birdcard.laideggs = 1
print(birdcard.laideggs, birdcard.egg_capacity)
print(birdcard.slash_food_cost)
html.appendChild(createBirdCardDiv(birdcard))

# Write to HTML
with open('test.html', 'w') as f:
    split = doc.toprettyxml().split('\n')
    preface = split[0]
    body = '\n'.join(split[1:])
    xml = '\n'.join([preface, f'\n<?xml-stylesheet type="text/css" href="{os.path.join(os.getcwd(), 'style.css')}"?>', body[:]])
    f.write(xml)

1 4
False
